In [ ]:
// Load the EnvConfigHelper class from our project
#load "./EnvConfigHelper.cs"

  // Load configuration from .env file
 // var modelId = "gpt-4.1";
  //var endPoint = "https://dis-azopen-ai.openai.azure.com/";
  //var apiKey = "";

    // Load configuration from .env file
  var modelId = EnvConfigHelper.modelId;
  var endPoint = EnvConfigHelper.endPoint;
  var apiKey = EnvConfigHelper.apiKey;



// Display loaded configuration (without sensitive data)
Console.WriteLine("🔑 Configuration loaded successfully!");
Console.WriteLine($"Azure OpenAI Endpoint: {endPoint}");
Console.WriteLine($"Deployment Name: {modelId}");
Console.WriteLine($"Deployment Name: {apiKey}");
Console.WriteLine($"API Key: {(string.IsNullOrEmpty(apiKey) ? "❌ Not loaded" : "✅ Loaded (hidden)")}");

✅ Loaded .env from: d:\Dev\VSC\semantic-kernel-in-action-1-fundamentals-3836112\src\04_05e\.env
🔑 Configuration loaded successfully!
Azure OpenAI Endpoint: https://dis-azopen-ai.openai.azure.com/
Deployment Name: gpt-4.1
Deployment Name: 7vddAIvhIyaYiOumOwzB2Anip91Y2JHlKjEt5y5JEAEry4nfbVQqJQQJ99BGACHYHv6XJ3w3AAABACOGaCg3
API Key: ✅ Loaded (hidden)


In [4]:
#r "nuget: Microsoft.SemanticKernel, *"
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using System.ComponentModel;



Installed Packages Microsoft.SemanticKernel, 1.61.0

In [5]:

public sealed class BookingPlugin
{
    // a list of available dates the user can book
    private List<DateTime> _available = new List<DateTime>
    {
        // add dates for the next 9 days, ignore the time part
        new DateTime(DateTime.Now.Year, DateTime.Now.Month, DateTime.Now.Day + 1, 0, 0, 0),
        new DateTime(DateTime.Now.Year, DateTime.Now.Month, DateTime.Now.Day + 2, 0, 0, 0),
        new DateTime(DateTime.Now.Year, DateTime.Now.Month, DateTime.Now.Day + 3, 0, 0, 0),
        new DateTime(DateTime.Now.Year, DateTime.Now.Month, DateTime.Now.Day + 4, 0, 0, 0),
        new DateTime(DateTime.Now.Year, DateTime.Now.Month, DateTime.Now.Day + 5, 0, 0, 0),
        new DateTime(DateTime.Now.Year, DateTime.Now.Month, DateTime.Now.Day + 6, 0, 0, 0),
        new DateTime(DateTime.Now.Year, DateTime.Now.Month, DateTime.Now.Day + 7, 0, 0, 0),
        new DateTime(DateTime.Now.Year, DateTime.Now.Month, DateTime.Now.Day + 8, 0, 0, 0),
        new DateTime(DateTime.Now.Year, DateTime.Now.Month, DateTime.Now.Day + 9, 0, 0, 0),
    };

    // a list of booked dates
    private List<DateTime> _booked = new List<DateTime>();

[KernelFunction, Description("Determines the next available date for a booking")]
public DateTime GetNextAvailableDate([Description("The date to start searching from")] DateTime startDate)
{
  // get the next available date
  return _available.FirstOrDefault(d => d > startDate);
}


[KernelFunction, Description("Allows the user to make a booking.")]
public string BookAppointment([Description("The date of the appointment")] DateTime date)
{
  // check if the date is available, ignore the time part
  date = date.Date;


if (_available.Contains(date))
{
  // remove the date from the available list
 _available.Remove(date);

  // add the date to the booked list
  _booked.Add(date);

  // return a confirmation message
  return $"Appointment booked for {date.ToShortDateString()}";
}
else
{
  // return an error message
  return "The selected date is not available";
}
}
}

In [6]:
public sealed class ComplaintPlugin
{
  // a list of complaints

  private List<string> _complaints = new List<string>();

  // lets the customer file a complaint

[KernelFunction, Description("Lets a customer file a complaint")]
public string FileComplaint([Description("The complaint message")] string message)
{
  // add the complaint to the list
  _complaints.Add(message);
  // return a confirmation message
  return "Complaint filed";
}


// lets the manager fetch all complaints
[KernelFunction, Description("Lets the hotel manager fetch all complaints")]
public string FetchComplaints()
{
  // return all complaints
  return string.Join(Environment.NewLine, _complaints);
}

}

In [ ]:




Console.WriteLine("Launching Semantic Kernel Sandpit");

//var builder = Kernel.CreateBuilder()
    //.AddOpenAIChatCompletion(modelId, apiKey);

var builder = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: modelId,
        endpoint: endPoint,
        modelId: modelId, // optional, but should match the model behind the deployment
        apiKey: apiKey
    );



builder.Plugins.AddFromType<BookingPlugin>();
builder.Plugins.AddFromType<ComplaintPlugin>();

Kernel kernel = builder.Build();

// Enable planning / automatic function calling
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions,
};

// Create chat history and system prompt
var history =
    new ChatHistory("You are professional customer service agent that handles bookings." +
    "If the person says 'I am a manager', the agent will let them access " +
    "fetch all complaints in the ComplaintPlugin.");

// Get chat completion service
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

// Start the conversation
Console.Write("User > ");

string? userInput;
while ((userInput = Console.ReadLine()) != null)
{
    // Add user input
    history.AddUserMessage(userInput);
    // Get the response from the AI
    var result = await chatCompletionService.GetChatMessageContentAsync(
        history,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel);

    // Print the results
    Console.WriteLine("Assistant > " + result);

    // Add the message from the agent to the chat history
    history.AddAssistantMessage(result.Content ?? string.Empty);

    // Get user input again
    Console.Write("User > ");
}

Launching Semantic Kernel Sandpit
User > Assistant > Hello! How can I assist you today with your booking or complaint needs?
User > Assistant > Hi! How can I help you today? If you need to make a booking or would like to file a complaint, just let me know!
User > Assistant > Hello! How can I assist you today? If you’d like to make a booking or need help with something else, please let me know.
User > Assistant > How can I assist you today? If you need help with bookings, complaints, or anything else, just let me know!
User > Assistant > It looks like you’ve sent a cancel command. Would you like to cancel a booking, or is there something else you’d like to do? Please provide more details so I can assist you further.
User > Assistant > It looks like you are working with C# code related to setting up a Semantic Kernel with Booking and Complaint plugins, and enabling function calling capabilities. Is there something specific you’d like to know or do in relation to this code? If you have an

In [13]:
await Execute();

Result: Today is Saturday, 02 August 2025.

Here's how a policeman and a scientist would respond in your situation:

Policeman: "Absolutely, citizen! What’s your question? Don’t worry, you’re not in trouble… unless you ask me about my favorite donut flavor. (It’s jelly, by the way. Strictly off the record.) Fire away—whether it’s about law, safety, or if you’re lost on your way to a community bake sale, I’m all ears and ready to serve."

Scientist: "Absolutely! Please provide the user question you'd like me to answer as if I were Scientific, and I'll channel his personality and style for the response."

If you need more specific guidance from either the policeman or scientist, feel free to ask!
